In [1]:
import os, sys
import numpy as np
import psycopg2

In [2]:
# obtain paths for each colored mask image directory

masks_dir_path = "../app/assets/images/2Masks"
mask_dirs = os.listdir(masks_dir_path)
mask_dirs = [os.path.join(masks_dir_path, el) for el in mask_dirs]
fil = [os.path.isdir(el) for el in mask_dirs]

from itertools import compress
mask_dirs = list(compress(mask_dirs, fil))
print mask_dirs

['../app/assets/images/2Masks/cyan', '../app/assets/images/2Masks/green', '../app/assets/images/2Masks/magenta', '../app/assets/images/2Masks/yellow']


In [3]:
# obtain names of each unique mask

masks = os.listdir(mask_dirs[0])
mask_names = [el.split("_")[0] for el in masks]
mask_names = list(set(mask_names))
mask_names.sort()
print mask_names[:5]


['Diencephalon -', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 'Diencephalon - Anterior preoptic dopaminergic cluster', 'Diencephalon - Anterior pretectum cluster of vmat2 Neurons', 'Diencephalon - Caudal Hypothalamus']


In [63]:
# check file size of a single mask image

example_mask_image = os.listdir(mask_dirs[0])[0]
example_mask_image = os.path.join(mask_dirs[0], example_mask_image)
print os.stat(example_mask_image).st_size

391


In [44]:
# prepare postgreSQL table

create_table_command = (
"""
    DROP TABLE masks;
    CREAT TABLE masks (
        mask_id SERIAL PRIMARY KEY,
        mask_name TEXT NOT NULL,
        mask_image_path TEXT NOT NULL,
        mask_image_size INT
    )
""")

In [45]:
# connect to zbrain2db postgreSQL database

conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [46]:
# create table masks
try:
    #cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print error
finally:
    if conn is not None:
        conn.close()

In [47]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [48]:
# for each mask image create an entry in postgreSQL database

for i, img in enumerate(masks): # masks = os.listdir(mask_dirs[0])
    mask_image_path = img
    mask_name = img.split("_")[0]
    mask_image_size = os.stat(os.path.join(mask_dirs[0], mask_image_path)).st_size
    
    insert_command = ("INSERT INTO masks (mask_id, mask_name, mask_image_path, mask_image_size) values (%i, '%s', '%s', %i)"
                    % (i, mask_name.replace("'", "''"), mask_image_path.replace("'", "''"), mask_image_size))
    
    try:
        cursor.execute(insert_command)
    except Exception, e:
        print repr(e)

In [49]:
cursor.close()
conn.commit()

In [61]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM masks ORDER BY mask_id')

In [62]:
# list entries

rows = cursor.fetchall()
for row in rows[0:10]:
    print row

(0, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice001.png', 391)
(1, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice002.png', 391)
(2, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice003.png', 391)
(3, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice004.png', 391)
(4, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice005.png', 391)
(5, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 'Diencephalon - Anterior group of the posterior tubercular vm

In [58]:
print len(rows)

40572
